In [ ]:
import csv
from pprint import pprint as pp
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import re
import country_converter as coco
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import wptools as wp

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from random import uniform
from bs4 import BeautifulSoup
import time
from statistics import median
import nltk
nltk.download('stopwords')

%matplotlib inline

#Data Source B: 
More artist informtion from Wikipedia and Kaggle - artist personal details

Step1a: Scrap artist's birthday, birth place from Wikipedia 

In [ ]:
# the above csv file was cleaned to enable the artist name match how it is recorded in Wikipedia
music = pd.read_csv('track artist audio features cleaned.csv')

# drop duplicate index column
music.drop('Unnamed: 0', axis=1, inplace=True)  

# isolate artist id and name
music = music[['artist_id', 'artist_name']]

# drop duplicate artist
music.drop_duplicates(inplace=True)
music

In [ ]:
# split the data into groups because wiki has a time limit for scraping
# this is the error for time limit: (28, 'Failed to connect to en.wikipedia.org port 443 after 21022 ms: Timed out')
# first group
music_first = music[:800].copy()

# second group
music_second = music[800:1600].copy()  

# third group
music_third = music[1601:2400].copy()

# fourth group
music_fourth = music[2400:].copy()

In [ ]:
# function for scraping singer_details and saving to a df
def scrape_infobox(df_name):
    '''
    This function scrapes the infobox of artists from wikipedia and 
    appends the data as new columns to the dataframe
    '''
    count = 0 
    for index, each_name in zip(df_name.index, df_name['artist_name']):
        count += 1
        print(str(count) + ": " + each_name)
        try:
            page = wp.page(each_name).get_parse()
            keys = ['name', 'members', 'current_members', 'birth_name', 'origin', 'birth_place', 'birth_date']
            if keys[0] in page.data['infobox'].keys():  # artist name
                df_name.loc[index, 'name']= page.data['infobox'][keys[0]]
            else:
                df_name.loc[index, 'name'] = np.nan

            if keys[1] in page.data['infobox'].keys():  # bands members name
                df_name.loc[index, 'members'] = page.data['infobox'][keys[1]]
            elif keys[2] in page.data['infobox'].keys():
                df_name.loc[index, 'members'] = page.data['infobox'][keys[2]]
            else:
                df_name.loc[index, 'members'] = np.nan

            if keys[1] in page.data['infobox'].keys():  # members for bands
                df_name.loc[index, 'band'] = 'True'
            elif keys[2] in page.data['infobox'].keys():
                df_name.loc[index, 'band'] = 'True'
            else:
                df_name.loc[index, 'band'] = 'False'

            if keys[3] in page.data['infobox'].keys():  # artist birth name
                df_name.loc[index, 'birth_name'] = page.data['infobox'][keys[3]]
            else:
                df_name.loc[index, 'birth_name'] = np.nan

            if keys[4] in page.data['infobox'].keys():  # nationality
                df_name.loc[index, 'birth_place'] = page.data['infobox'][keys[4]]
            elif keys[5] in page.data['infobox'].keys():
                df_name.loc[index, 'birth_place'] = page.data['infobox'][keys[5]]
            else:
                df_name.loc[index, 'birth_place'] = np.nan

            if keys[6] in page.data['infobox'].keys():  # age
                df_name.loc[index, 'birth_date'] = page.data['infobox'][keys[6]]
            else:
                df_name.loc[index, 'birth_date'] = np.nan

        except AttributeError as err:
            pass
        except TypeError as err:
            pass
        except LookupError as err1:
            pass
        except ValueError as err2:
            pass   

    return

In [ ]:
# scrape data for artist groups individually 
music_first = scrape_infobox(df_name=music_first)
music_second = scrape_infobox(df_name=music_second)
music_third = scrape_infobox(df_name=music_third)
music_fourth = scrape_infobox(df_name=music_fourth)

In [ ]:
# combine the artist groups vertically into one dataframe
artist_details_combined = pd.concat([music_first, music_second, music_third, music_fourth])

Actually, the scraping shouldnt be done in the below manner, i.e saving to four different csv. so we wrote a code(above ) that scrapes then combines the df into one before saving to a csv. (If we leave this here we will be shooting ourselves in the leg)

In [ ]:
# #The artist details were saved to 4 different files due to wikipedia's respond time.
# df1 = pd.read_csv('artist1.csv', encoding='utf-8')
# df2 = pd.read_csv('artist2.csv', encoding='utf-8')
# df3 = pd.read_csv('artist3.csv', encoding='utf-8')
# df4 = pd.read_csv('artist4.csv', encoding='utf-8')

# #To vertically combine all the files
# df_artist = pd.concat([df1, df2, df3, df4], axis=0)
# #save the artist details to a file.
# df_artist.to_csv('artist_details_combined.csv', index=False)

(Step1c): Name clashes on different artists was discovered later, by that time we already started scraping artists' detail from Wikipedia, hence we take the drop_list value from our previous cell and repeat this process with artist_details_combined.csv 

Name the file artist_details_unique_artist_name.csv

In [ ]:
# drop_list from previous cells, the value was printed and stored, so that we can use it without running the code again
drop_list = ['7lf4M8KM02hIiVGHKsAl2o', '6h9telSBnbabV6eyeQtWO7', '0mZsmXegjYM7hNw0QGE5b7', '2uFUBdaVGtyMqckSeCl0Qj', '68abRTdO4meYReMWHvBYb0', '02J7bo06ZZ1XmSGochlWak', '0hDmcvucqFYoojOpHfDiZ0', '4pchWejl1hLpJ8zXvZlqLb', '6GDDxhe281I5aNSgCYphex', '7n2Ycct7Beij7Dj7meI4X0', '7EXJJpgDl7y4szvrpUYj0s', '3a0gvZAydt98HKVbGb0V67', '0hEurMDQu99nJRq8pTxO14', '0ZkhUsUWbgsB356AUiiIWz', '3TOqt5oJwL9BE2NG9MEwDa', '2m1l9MLSslzup4vvokKgvQ', '37SQDDbylULQMBCICskLdA', '536pTabXNhaK7O7VfjOk7P', '5o7lbGxFCLde3JqX2EKzjZ', '0a1pOwbTl7KjPdm9h7DJT5', '2yp6zqk49KOKKrOSSsUb75', '3EAHF3jdnHHdko5DBrhRUP']
df = pd.read_csv('artist_details_combined.csv', encoding='utf-8')
for row in df.index:
    if df.loc[row, "artist_id"] in drop_list:
        df.drop(row, inplace=True)
df.to_csv('artist_details_unique_artist_name.csv', index=False)

Step2: Combine gender dataset from Kaggle with artist details from Wikipedia

In [ ]:
# read the gender data downloaded from Kaggle
gender_df = pd.read_csv('singers_gender.csv', encoding='latin1')

# read the artist details with unique artist name csv
artist = pd.read_csv('artist_details_unique_artist_name.csv')

In [ ]:
# combine the above datasets on the artist_name column using a left join
artists_detail_with_category = artist.merge(gender_df, how='left', left_on='artist_name', right_on='artist')

# save the above df to csv
artists_detail_with_category.to_csv('artists_detail_with_category.csv', index=False)

Step3: Organising the data

We need to convert birth place information into birth country, and convert birthday into birth year.

Country:

We will be using a python package called country_converter (coco) to extract country information. 

However, some astist's country information is missing on Wikipedia but available in the Kaggle dataset. The information from Wikipedia could be unorganised, e.g. for multicultural artists the way Wikipedia store the information could be inconsistent, for many artists from the US it's showing the state but not the country name. 

There is not a python package that could fetch city or state information to convert it into country, so we extract country information from Kaggle (column 'category') first, and overwrite it to the data from Wikipedia (column 'birth_place'). Then we go through column 'birth_place'.

Birth year:

To simplify the process, we convert birth_date to birth_year at the same time. 




Step3a: 


Convert birth_date to birth_year. 


Overwrite country information extracted from Kaggle to Wikipedia 


Save the process into artist_category_to_more.csv 


In [ ]:
df = pd.read_csv('artists_detail_with_category.csv', encoding='utf-8')
df = df.sort_values(by=['category'], ascending=True)

for row in df.index:
    artist_category = str(df.loc[row, 'category'])
    if artist_category == 'nan':
        # print('break')
        break
    else:
        
        # some of the variations and counties can't be recognized by country_converter
        if 'American' in artist_category:
            df.loc[row, 'birth_place'] = 'United States'
        elif 'Canadian' in artist_category:
            df.loc[row, 'birth_place'] = 'Canada'
        elif any(c in artist_category for c in ('British', 'English', 'Scottish', 'Welsh')):
            df.loc[row, 'birth_place'] = 'United Kingdom'
            print('uk')
        else:
            artist_country = coco.convert(artist_category, to='name_short')
            if artist_country != 'not found':
                df.loc[row, 'birth_place'] = artist_country
                print(artist_country)

#to get only the birth year of the artists
df['birth_year'] = df['birth_date'].str.extract(r'(\d{4})')

df.to_csv('artist_category_to_more.csv', index=False)

Step3b: Extract all the country information and store it in artist_country column 


Save it in artists_details_final.csv

In [ ]:
df = pd.read_csv('artist_category_to_more.csv', encoding='utf-8')
artist_country_dic = {}

birth_place_list = df['birth_place'].tolist()
birth_place_list = list(set(birth_place_list))

for birth_place in birth_place_list:
    artist_country = []
    bp = re.sub('[^A-Za-z ,]+', '', str(birth_place)).split(',')
    for element_raw in bp:
        element = element_raw.strip()
        
        # some of the variations and counties can't be recognized by country_converter  
        if element in ('England', 'Scotland', 'Wales', 'Northern Ireland', 'UK'):
            artist_country.append('United Kingdom')
        elif element == 'U.S.':
            artist_country.append('United State')
        else:
            country = coco.convert(element, to='name_short')
            if country != 'not found':
                if type(country) == list:
                    for c in country:
                        artist_country.append(c)
                else:
                    artist_country.append(country)

    if len(artist_country) > 1:
        artist_country_cleaned = []
        for ac in artist_country:
            if ac not in artist_country_cleaned:
                artist_country_cleaned.append(ac)
                
        # remove error due to (comonents in) city/state name clashes country name
        if 'United States' in artist_country_cleaned:
            if 'Georgia' in artist_country_cleaned:
                artist_country_cleaned.remove('Georgia')
            if 'Jersey' in artist_country_cleaned:
                artist_country_cleaned.remove('Jersey')
        if 'Trinidad and Tobago' in artist_country_cleaned:
            if 'Spain' in artist_country_cleaned:
                artist_country_cleaned.remove('Spain')
        artist_country = artist_country_cleaned

    if len(artist_country) == 1:
        artist_country_dic[birth_place] = artist_country[0]
    elif len(artist_country) == 0:
        artist_country_dic[birth_place] = 'N/A'
    else:
        artist_country_dic[birth_place] = 'multicultural'

df['artist_country'] = df['birth_place'].map(artist_country_dic)

df.to_csv('artists_details_final.csv', index=False)